# 0. Информация об видеокарте

In [1]:
!nvidia-smi

Sat Apr 13 22:28:04 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 546.12                 Driver Version: 546.12       CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3080      WDDM  | 00000000:01:00.0  On |                  N/A |
| 59%   54C    P0             121W / 370W |   1828MiB / 10240MiB |      4%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

True

# 1. Загрузка библиотек и объявление путей

In [1]:
import os
import glob
import random
import cv2
import pandas as pd
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm
import json
from pprint import pprint
import shutil
import yaml
from ultralytics import YOLO

import warnings
warnings.filterwarnings("ignore")

In [3]:
# Пути до датасетов с автомобилями
PATH_CAR = './DATASET/CarDetection' # Путь до датасета Car Detection
PATH_CARS = './DATASET/CarsDetection/Cars Detection/' # Путь до датасета Cars Detection
PATH_CAR_PERSON = './DATASET/CarPerson/Car-Person-v2-Roboflow-Owais-Ahmad/' # Путь до датасета Car Person

# Пути до датасетов с ЛЭП
PATH_STN = './DATASET/STN' # Путь до датасета STN PLAD
PATH_UTILLUTY_POLE = './DATASET/UtilityPole/' # Путь до датасета Utiluty Pole
PATH_TRANSMISSION = './DATASET/Transmission/data_original_size_v1/data_original_size/' # Путь до датасета Transmission

# Пути до датасетов с деревьями
PATH_TREE = './DATASET/Tree/' # Путь до датасета Tree Detection
PATH_TREE_DETECTION = './DATASET/TreeDetection/' # Путь до датасета Tree Detection
PATH_TREE_100 = './DATASET/Trees100/' # Путь до датасета Tree 100

# Пути до датасетов с птицами
PATH_BIRD_ANNOTATION = './DATASET/BirdAnnotation/' # Путь до датасета Bird Annotation 
PATH_BIRDS_DETECTOR = './DATASET/BirdsDetector/' # Путь до датасета Birds Detector
PATH_BIRDS_V3 = './DATASET/BirdsV3/' # Путь до датасета BirdsV3

# Пути до датасетов со зданиями
PATH_BUILDING_DETECTION = './DATASET/BuildingDetection/' # Путь до датасета Building Detection
PATH_JOSH_EDITS = './DATASET/JoshEdits/' # Путь до датасета JoshEdits

# Словарь по понижению размрености датасетов
DATASET_DOWNGRADES = {
    'dataCarDetection': 2, 
    'dataCarPerson': 3, 
    'dataCarsDetection': 4,
    'dataSTN': 1, 
    'dataUnilityPole': 1, 
    'dataTransmission': 1,
    'dataTree': 2, 
    'dataTreeDetection': 2, 
    'dataTree100': 2,
    'dataBirdAnnotation': 4, 
    'dataBirdsDetectod': 4, 
    'dataBirdsV3': 2,
    'dataBuildingDetection': 2, 
    'dataJoshEdits': 2
}

In [4]:
def print_variable_name(variable):
    '''
        Функция позволяет получить имя переменной
        Input: 
            variable - переменная
        Output:
            variable_name - имя переменной
    '''
    variable_name = [name for name, value in globals().items() if value is variable][-1]
    return variable_name

# 2. Обработка данных

In [5]:
def converterYoloToDataFrame(labels_list, path, classes):
    '''
        Функция позволяет перенести все данные с формата YOLO в DataFrame
        Input:
            labels_list - набор названий изображений
            path - путь до папки с изображениями
    '''
    labels = {} # Словарь объктов
    number_item = 0 # Номер записи
    for item in tqdm(labels_list):
        # Читаем все записи
        with open(path + item) as f:
            lines = f.readlines()
    
        # Проходимся по каждой записи
        for line in lines:
            # Получаем координаты в процентах
            x_center, y_center, w, h = (float(number) for number in line.split()[1:]) 
    
            # Заносим в словарь
            labels[number_item] = {
                'image_name': '/'.join(path.split('/')[:-2]) + '/images/' + item[:-3] + 'jpg',
                'x_center': x_center, 
                'y_center': y_center, 
                'w': w, 
                'h': h,
                'classes': classes
            }
            number_item += 1
    return labels

In [6]:
def datasetMinimization(dataSet, count):
    '''
        Функция позволяет понизить размер DataFrame за счёт удаления фотографий
        Input:
            dataSet - первоначальный DataFrame
            count - количество, во сколько раз сжать DataFrame
    '''
    # Список во всеми именами фотографий
    list_image_name = dataSet['image_name'].unique().tolist()

    # Кол-во фотографий в начале
    start_count_image = len(list_image_name)
    
    # Получаем именя изображений, которые оставляем
    name_file = random.sample(list_image_name, len(list_image_name) // count)

    # Записи с этими файлами
    dataSet = dataSet[dataSet['image_name'].isin(name_file)]

    # Кол-во фотографий после обработки
    end_count_image = len(dataSet['image_name'].unique())
    
    # Выведем системноо сообщение об результате сокращения фотографий
    print(f'There were {start_count_image} photos, now there are {end_count_image}')
    
    return dataSet

## 2.0 Классы для модели
    0 - машина
    1 - ЛЭП
    2 - Деревья
    3 - Птицы
    4 - Дом/здание
    5 - Провода

## 2.1. Датасеты с автомобилями
### 2.1.1 Обработка датасета Car Detection

In [7]:
img_h, img_w = (380, 676) # Размеры изображения
dataCarDetection = pd.read_csv(PATH_CAR + '/train_solution_bounding_boxes (1).csv') # Читаем датасет

dataCarDetection.rename(columns={'image':'image_name'}, inplace=True) # Переименовываем колонку

dataCarDetection.sample(10) # посмотрим на данные

,image_name,xmin,ymin,xmax,ymax
378,vid_4_29940.jpg,40.599132,199.750643,102.720695,225.181789
455,vid_4_6400.jpg,346.315485,190.458494,445.612156,218.824003
402,vid_4_3340.jpg,527.788712,175.786679,630.998553,219.802124
331,vid_4_26480.jpg,466.156295,179.699163,550.778582,220.291184
470,vid_4_700.jpg,586.486252,181.166345,676.000000,248.167632
384,vid_4_29960.jpg,0.000000,195.838160,45.979740,231.539575
379,vid_4_29940.jpg,95.383502,194.860039,164.353111,227.627091
100,vid_4_13860.jpg,609.476122,173.616474,676.000000,216.378700
74,vid_4_13680.jpg,260.225760,183.611647,279.302460,202.195946
419,vid_4_6200.jpg,241.149059,181.655405,361.968162,227.627091


In [8]:
# Получим координаты центра объекта в процентах 
dataCarDetection['x_center'] = (dataCarDetection['xmin'] + dataCarDetection['xmax']) / 2 / img_w 
dataCarDetection['y_center'] = (dataCarDetection['ymin'] + dataCarDetection['ymax']) / 2 / img_h 

# Получим размеры объекта в процентах
dataCarDetection['w'] = (dataCarDetection['xmax'] - dataCarDetection['xmin']) / img_w # Получаем ширину объекта в процентах
dataCarDetection['h'] = (dataCarDetection['ymax'] - dataCarDetection['ymin']) / img_h # Получаем ширину объекта в процентах
dataCarDetection['classes'] = 0 # Задаём класс объекта
dataCarDetection.drop(
    ['xmin', 'xmax', 'ymin', 'ymax'], # Удаляем ненужные признаки
    axis=1, inplace=True
)

# Изменим путь до файла
dataCarDetection['image_name'] = f'{PATH_CAR}/training_images/' + dataCarDetection['image_name']

# Очистим память 
del img_h, img_w

# Уменьшаем размер DataFrame
dataCarDetection = datasetMinimization(dataCarDetection, DATASET_DOWNGRADES[print_variable_name(dataCarDetection)])

dataCarDetection.sample(10) # Посмотрим на данные

There were 355 photos, now there are 177


,image_name,x_center,y_center,w,h,classes
358,./DATASET/CarDetection/training_images/vid_4_2...,0.034009,0.557835,0.068017,0.087516,0
497,./DATASET/CarDetection/training_images/vid_4_9...,0.065186,0.559464,0.095412,0.085492,0
346,./DATASET/CarDetection/training_images/vid_4_2...,0.056078,0.564270,0.112156,0.100386,0
141,./DATASET/CarDetection/training_images/vid_4_1...,0.090087,0.576496,0.130970,0.101673,0
147,./DATASET/CarDetection/training_images/vid_4_1...,0.178726,0.586792,0.215630,0.142857,0
383,./DATASET/CarDetection/training_images/vid_4_2...,0.376990,0.555904,0.096961,0.068211,0
414,./DATASET/CarDetection/training_images/vid_4_6...,0.513748,0.552043,0.179450,0.117117,0
225,./DATASET/CarDetection/training_images/vid_4_2...,0.026049,0.578427,0.052098,0.074646,0
474,./DATASET/CarDetection/training_images/vid_4_8...,0.742535,0.529024,0.184268,0.107513,0
460,./DATASET/CarDetection/training_images/vid_4_6...,0.698625,0.530808,0.167149,0.113256,0


In [9]:
import inspect
xer = 0

print_variable_name(dataCarDetection)

'dataCarDetection'

## 2.1.2. Обработка датасета Cars Detection

In [10]:
dataCarsDetection = pd.DataFrame(columns=dataCarDetection.columns) 

# Проходимся по всем выборкам
for sample in ['train', 'valid', 'test']:
    labels_name = os.listdir(f'{PATH_CARS}{sample}/labels') # Список файлов с разметкой 
    labels = pd.DataFrame(converterYoloToDataFrame(labels_name, f'{PATH_CARS}{sample}/labels/', 0)) # Получаем датасет с разметкой
    dataCarsDetection = pd.concat([dataCarsDetection, labels.T]) # Соединяем выборки в единый датасет

del labels_name, labels # Очищаем память

# Уменьшаем размер DataFrame
dataCarsDetection = datasetMinimization(dataCarsDetection, DATASET_DOWNGRADES[print_variable_name(dataCarsDetection)])

dataCarsDetection.sample(10) # Посмотрим на данные

  0%|          | 0/878 [00:00<?, ?it/s]

  0%|          | 0/250 [00:00<?, ?it/s]

  0%|          | 0/126 [00:00<?, ?it/s]

There were 1254 photos, now there are 313


,image_name,x_center,y_center,w,h,classes
171,./DATASET/CarsDetection/Cars Detection/test/im...,0.486779,0.518029,0.759615,0.961538,0
597,./DATASET/CarsDetection/Cars Detection/train/i...,0.507212,0.483173,0.899038,0.736779,0
518,./DATASET/CarsDetection/Cars Detection/train/i...,0.663462,0.729567,0.579327,0.461538,0
447,./DATASET/CarsDetection/Cars Detection/train/i...,0.75601,0.674279,0.108173,0.075721,0
680,./DATASET/CarsDetection/Cars Detection/train/i...,0.491587,0.58774,0.983173,0.822115,0
93,./DATASET/CarsDetection/Cars Detection/valid/i...,0.768029,0.602163,0.459135,0.439904,0
1381,./DATASET/CarsDetection/Cars Detection/train/i...,0.457933,0.295673,0.855769,0.557692,0
628,./DATASET/CarsDetection/Cars Detection/train/i...,0.646635,0.419471,0.03125,0.014423,0
1403,./DATASET/CarsDetection/Cars Detection/train/i...,0.301683,0.776442,0.135817,0.16226,0
285,./DATASET/CarsDetection/Cars Detection/train/i...,0.512019,0.55649,0.883413,0.53125,0


### 2.1.3. Обработка датасета Car Person

In [11]:
dataCarPerson = pd.DataFrame(columns=dataCarDetection.columns)
for sample in ['train', 'valid', 'test']:
    labels_name = os.listdir(f'{PATH_CAR_PERSON}{sample}/labels') # Список файлов с разметкой
    labels = pd.DataFrame(converterYoloToDataFrame(labels_name, f'{PATH_CAR_PERSON}{sample}/labels/', 0)) # Получаем датасет с разметкой
    dataCarPerson = pd.concat([dataCarPerson, labels.T]) # Соединяем выборки в единый датасет
    
del labels_name, labels # Очищаем память

# Уменьшаем размер DataFrame
dataCarPerson = datasetMinimization(dataCarPerson, DATASET_DOWNGRADES[print_variable_name(dataCarPerson)])

dataCarPerson.sample(10) # Посмотрим на данные

  0%|          | 0/1571 [00:00<?, ?it/s]

  0%|          | 0/448 [00:00<?, ?it/s]

  0%|          | 0/224 [00:00<?, ?it/s]

There were 2243 photos, now there are 747


,image_name,x_center,y_center,w,h,classes
6655,./DATASET/CarPerson/Car-Person-v2-Roboflow-Owa...,0.504808,0.515625,0.395433,0.438702,0
258,./DATASET/CarPerson/Car-Person-v2-Roboflow-Owa...,0.645433,0.487981,0.069712,0.1875,0
7326,./DATASET/CarPerson/Car-Person-v2-Roboflow-Owa...,0.298077,0.283654,0.180288,0.128606,0
807,./DATASET/CarPerson/Car-Person-v2-Roboflow-Owa...,0.183894,0.689904,0.138221,0.425481,0
3169,./DATASET/CarPerson/Car-Person-v2-Roboflow-Owa...,0.692308,0.466346,0.020433,0.057692,0
4931,./DATASET/CarPerson/Car-Person-v2-Roboflow-Owa...,0.456731,0.765625,0.058894,0.128606,0
7590,./DATASET/CarPerson/Car-Person-v2-Roboflow-Owa...,0.355769,0.311298,0.044471,0.097356,0
11616,./DATASET/CarPerson/Car-Person-v2-Roboflow-Owa...,0.40625,0.536058,0.058894,0.219952,0
2071,./DATASET/CarPerson/Car-Person-v2-Roboflow-Owa...,0.435096,0.829327,0.106971,0.302885,0
6846,./DATASET/CarPerson/Car-Person-v2-Roboflow-Owa...,0.21875,0.478365,0.100962,0.11899,0


## 2.2. Обработка датасет с ЛЭП
### 2.2.1 Обработка датасета STN_PLAD

In [12]:
# Читаем файл с разметкой датасета STN
with open(PATH_STN + '/annotations.json') as f:
    dataSTN = json.load(f)

pprint(dataSTN['categories']) # Посмотрим на категории

# Получим два DataFrame, один с изображениями, другой с разметкой объектов
data = pd.DataFrame(dataSTN['images']).rename(columns={'id': 'image_id'})
data1 =  pd.DataFrame(dataSTN['annotations'])

# Объединим два дасатсета и удалим лишние признаки
dataSTN = pd.merge(data, data1, on='image_id', how='left').drop(
    [
        'licence', 'segmentation', 'date_captured', 'coco_url', 
        'iscrowd', 'image_id'
    ], axis=1
)

# Очистим память 
del data, data1 

dataSTN.sample(10) # Посмотрим на данные

[{'id': 0, 'name': 'tower', 'supercategory': 'tower'},
 {'id': 1, 'name': 'insulator', 'supercategory': 'component'},
 {'id': 2, 'name': 'spacer', 'supercategory': 'component'},
 {'id': 3, 'name': 'damper', 'supercategory': 'component'},
 {'id': 5, 'name': 'plate', 'supercategory': 'component'}]


,file_name,height,width,flickr_url,area,bbox,category_id,id
1954,20181127-A1/DJI_0450.JPG,3078,5472,,2680,"[2762, 2281, 40, 67]",3,1962
1045,20181127-A1/DJI_0417.JPG,3078,5472,,2376,"[4831, 1485, 66, 36]",3,1051
661,20181127-A1/DJI_0475.JPG,3078,5472,,855,"[2524, 1857, 19, 45]",3,664
1344,20181129-R/DJI_0025.JPG,3648,5472,,2499,"[248, 2700, 49, 51]",3,1351
1081,20181127-A1/DJI_0453.JPG,3078,5472,,1271,"[4200, 1975, 31, 41]",3,1087
1147,20181127-A1/DJI_0509.JPG,3078,5472,,4032,"[4964, 946, 63, 64]",2,1153
40,20181129-R/DJI_0024.JPG,3648,5472,,231264,"[210, 1378, 292, 792]",1,41
168,20181127-A1/DJI_0452.JPG,3078,5472,,3078,"[2621, 1432, 114, 27]",5,169
625,20181127-A1/DJI_0498.JPG,3078,5472,,7905062,"[799, 1001, 3806, 2077]",0,628
2398,20181127-A1/DJI_0508.JPG,3078,5472,,2378,"[1089, 628, 58, 41]",3,2411


In [13]:
# Получим координаты объекта
dataSTN[['xmin', 'ymin', 'xmax', 'ymax']] = dataSTN['bbox'].apply(lambda x: pd.Series(x))
dataSTN['xmax'] = dataSTN['xmax'] + dataSTN['xmin']
dataSTN['ymax'] = dataSTN['ymax'] + dataSTN['ymin']

# Получим координаты центра объекта в процентах 
dataSTN['x_center'] = (dataSTN['xmin'] + dataSTN['xmax']) / 2 / dataSTN['width']
dataSTN['y_center'] = (dataSTN['ymin'] + dataSTN['ymax']) / 2 / dataSTN['height']

# Получим размеры объекта в процентах
dataSTN['w'] = (dataSTN['xmax'] - dataSTN['xmin']) / dataSTN['width']
dataSTN['h'] = (dataSTN['ymax'] - dataSTN['ymin']) / dataSTN['height']

# Переименуем признаки
dataSTN.rename(
    columns={
        'file_name': 'image_name',
        'category_id': 'classes'
    }, 
    inplace=True
) 

# Удалим ненужные признаки
dataSTN.drop(
    [
        'height', 'width', 'flickr_url',
        'area', 'bbox', 'id', 'xmin', 
        'xmax', 'ymin', 'ymax'
    ], 
    axis=1, inplace=True
)

# Изменим путь до файла
dataSTN['image_name'] = './DATASET/STN/' + dataSTN['image_name']

# Оставим фотографии принадлижащие только к опоре ЛЭП
dataSTN = dataSTN[dataSTN['classes'] == 0]
dataSTN['classes'] = dataSTN['classes'] + 1

dataSTN.sample(10) # Посомтрим на данные

,image_name,classes,x_center,y_center,w,h
302,./DATASET/STN/20181127-A1/DJI_0429.JPG,1,0.496071,0.658057,0.074744,0.683886
167,./DATASET/STN/20181127-A1/DJI_0452.JPG,1,0.502558,0.680799,0.542398,0.638402
1839,./DATASET/STN/20181127-A1/DJI_0520.JPG,1,0.392361,0.500162,0.223319,0.999675
863,./DATASET/STN/20181127-A1/DJI_0492.JPG,1,0.465278,0.674951,0.555556,0.650097
735,./DATASET/STN/20181127-A1/DJI_0434.JPG,1,0.465826,0.436972,0.099050,0.873294
2262,./DATASET/STN/20181127-A1/DJI_0515.JPG,1,0.706414,0.683398,0.526133,0.633203
1540,./DATASET/STN/20181127-A1/DJI_0472.JPG,1,0.303728,0.651559,0.470760,0.696881
2322,./DATASET/STN/20181129-R/DJI_0023.JPG,1,0.495340,0.635691,0.919408,0.728618
1345,./DATASET/STN/20181127-A1/DJI_0507.JPG,1,0.383681,0.724334,0.474598,0.551332
441,./DATASET/STN/20181129-R/DJI_0020.JPG,1,0.839364,0.410636,0.017909,0.110197


### 2.2.2. Обработка датасета Utility Pole

In [14]:
dataUnilityPole = pd.DataFrame(columns=dataCarDetection.columns) 

# Проходимся по всем выборкам
for sample in ['train', 'test']:
    labels_name = os.listdir(f'{PATH_UTILLUTY_POLE}{sample}/labels') # Список файлов с разметкой 
    labels = pd.DataFrame(converterYoloToDataFrame(labels_name, f'{PATH_UTILLUTY_POLE}{sample}/labels/', 1)) # Получаем датасет с разметкой
    dataUnilityPole = pd.concat([dataUnilityPole, labels.T]) # Соединяем выборки в единый датасет

del labels_name, labels # Очищаем память
dataUnilityPole.sample(10) # Посмотрим на данные

  0%|          | 0/179 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

,image_name,x_center,y_center,w,h,classes
45,./DATASET/UtilityPole/train/images/a93a35ee792...,0.776442,0.629808,0.091346,0.307692,1
86,./DATASET/UtilityPole/train/images/distributio...,0.6875,0.776442,0.072115,0.259615,1
71,./DATASET/UtilityPole/train/images/Cisco_Rural...,0.328125,0.655048,0.064904,0.329327,1
65,./DATASET/UtilityPole/train/images/b5c25646-fc...,0.530048,0.485577,0.127404,0.951923,1
57,./DATASET/UtilityPole/train/images/b3978835746...,0.729567,0.481971,0.266827,0.742788,1
154,./DATASET/UtilityPole/train/images/images--8-_...,0.473558,0.449519,0.221154,0.899038,1
180,./DATASET/UtilityPole/train/images/istockphoto...,0.498798,0.503606,0.175481,0.555288,1
186,./DATASET/UtilityPole/train/images/istockphoto...,0.175481,0.75,0.052885,0.173077,1
150,./DATASET/UtilityPole/train/images/images--7-_...,0.545673,0.501202,0.254808,0.949519,1
7,./DATASET/UtilityPole/test/images/utilitypolea...,0.52524,0.513221,0.262019,0.78125,1


### 2.2.3. Обработка датасета Transmission

In [15]:
list_image = glob.glob(PATH_TRANSMISSION + '*.jpg')
labels = {} # Словарь объктов
number_item = 0 # Номер записи
for path_image in tqdm(list_image):
    with open(path_image[:-3] + 'json') as f:
        json_file = json.load(f)
    xmax = 0
    xmin = 2**10
    ymax = 0
    ymin = 2**10
    height = json_file['imageHeight']
    width = json_file['imageWidth']
    for item in json_file['shapes']:
        if item['label'] in ['tower_wooden', 'cable']:
            for data in item['points']:
                x, y = data
                xmax = x if x > xmax else xmax
                ymax = y if x > ymax else ymax
                xmin = x if y < xmin else xmin
                ymin = y if y < ymin else ymin
            x_center = (xmin + xmax) / 2 / width
            y_center = (ymin + ymax) / 2 / height
            w = (xmax - xmin) / width
            h = (ymax - ymin) / height
            labels[number_item] = {
                'image_name': path_image,
                'x_center': 0.0 if x_center < 0.0 else 1.0 if x_center > 1.0 else x_center, 
                'y_center': 0.0 if y_center < 0.0 else 1.0 if y_center > 1.0 else x_center, 
                'w': 0.0 if w < 0.0 else 1.0 if w > 1.0 else w, 
                'h': 0.0 if h < 0.0 else 1.0 if h > 1.0 else h,
                'classes': 1 if item['label'] == 'tower_wooden' else 5
            }
            number_item += 1
            
dataTransmission = pd.DataFrame(labels).T # Переносим все в DataFrame

# Очищаем память
del list_image, labels, number_item, xmin, xmax, ymin, ymax, h, w, json_file 

# Уменьшаем размер DataFrame
dataTransmission = datasetMinimization(dataTransmission, DATASET_DOWNGRADES[print_variable_name(dataTransmission)])

dataTransmission.sample(10) # Посмотрим на данные

  0%|          | 0/1242 [00:00<?, ?it/s]

There were 1231 photos, now there are 1231


,image_name,x_center,y_center,w,h,classes
8949,./DATASET/Transmission/data_original_size_v1/d...,0.500007,0.500007,1.0,0.622222,5
9232,./DATASET/Transmission/data_original_size_v1/d...,0.420286,0.420286,0.65379,0.704981,5
10014,./DATASET/Transmission/data_original_size_v1/d...,0.468277,0.468277,0.193182,0.413826,5
1456,./DATASET/Transmission/data_original_size_v1/d...,0.500142,0.500142,0.997763,0.401957,5
7883,./DATASET/Transmission/data_original_size_v1/d...,0.49987,0.49987,0.99974,0.411572,5
3640,./DATASET/Transmission/data_original_size_v1/d...,0.499512,0.499512,0.999023,0.0,5
7352,./DATASET/Transmission/data_original_size_v1/d...,0.978784,0.978784,0.041911,0.263369,5
489,./DATASET/Transmission/data_original_size_v1/d...,0.546646,0.546646,0.906048,0.927882,5
8729,./DATASET/Transmission/data_original_size_v1/d...,0.962318,0.962318,0.074843,0.946552,5
4238,./DATASET/Transmission/data_original_size_v1/d...,0.817057,0.817057,0.365335,0.334869,5


## 2.3. Обработка датасетов с деревьями
### 2.3.1. Обработка датасета Tree Object Detecion

In [16]:
dataTree = pd.DataFrame(columns=dataCarDetection.columns)
labels_name = os.listdir(f'{PATH_TREE}train/labels/') # Список файлов с разметкой
labels = pd.DataFrame(converterYoloToDataFrame(labels_name, f'{PATH_TREE}train/labels/', 2)) # Получаем датасет с разметкой
dataTree = pd.concat([dataTree, labels.T]) # Соединяем выборки в единый датасет
    
del labels_name, labels # Очищаем память

dataTree.sample(10) # Посмотрим на данные

  0%|          | 0/214 [00:00<?, ?it/s]

,image_name,x_center,y_center,w,h,classes
630,./DATASET/Tree/train/images/tree-cutout_png_jp...,0.5,0.5,1.0,1.0,2
169,./DATASET/Tree/train/images/big-tree-pixabay_j...,0.698438,0.966406,0.085938,0.067187,2
74,./DATASET/Tree/train/images/20230709_155609_jp...,0.7375,0.240625,0.175,0.398438,2
605,./DATASET/Tree/train/images/The-Survivor-Tree-...,0.095312,0.492188,0.190625,0.6,2
224,./DATASET/Tree/train/images/draw-trees_fb-img_...,0.682813,0.453906,0.634375,0.667188,2
514,./DATASET/Tree/train/images/RGXRFD_jpg.rf.f024...,0.680469,0.714063,0.104688,0.13125,2
215,./DATASET/Tree/train/images/big-tree-pixabay_j...,0.901563,0.899219,0.05,0.185938,2
386,./DATASET/Tree/train/images/images-61-_jpg.rf....,0.503125,0.517969,0.99375,0.851562,2
264,./DATASET/Tree/train/images/images-26-_jpg.rf....,0.5,0.5,1.0,1.0,2
470,./DATASET/Tree/train/images/niko-photos-tGTVxe...,0.936719,0.860156,0.079687,0.08125,2


### 2.3.2. Обработка датасета TreeDetection 

In [17]:
dataTreeDetection = pd.DataFrame(columns=dataCarDetection.columns) 

# Проходимся по всем выборкам
for sample in ['train', 'valid', 'test']:
    labels_name = os.listdir(f'{PATH_TREE_DETECTION}{sample}/labels') # Список файлов с разметкой 
    labels = pd.DataFrame(converterYoloToDataFrame(labels_name, f'{PATH_TREE_DETECTION}{sample}/labels/', 2)) # Получаем датасет с разметкой
    dataTreeDetection = pd.concat([dataTreeDetection, labels.T]) # Соединяем выборки в единый датасет

del labels_name, labels # Очищаем память

# Уменьшаем размер DataFrame
dataTreeDetection = datasetMinimization(dataTreeDetection, DATASET_DOWNGRADES[print_variable_name(dataTreeDetection)])

dataTreeDetection.sample(10) # Посмотрим на данные

  0%|          | 0/1803 [00:00<?, ?it/s]

  0%|          | 0/174 [00:00<?, ?it/s]

  0%|          | 0/86 [00:00<?, ?it/s]

There were 2060 photos, now there are 1030


,image_name,x_center,y_center,w,h,classes
255,./DATASET/TreeDetection/valid/images/90011760-...,0.51875,0.511719,0.934375,0.8625,2
335,./DATASET/TreeDetection/train/images/210407_17...,0.664844,0.378906,0.66875,0.757812,2
726,./DATASET/TreeDetection/train/images/211011_14...,0.436719,0.417969,0.43125,0.696094,2
103,./DATASET/TreeDetection/test/images/4e5178d0-2...,0.498437,0.496094,0.714063,0.936719,2
95,./DATASET/TreeDetection/valid/images/221009_16...,0.7375,0.485938,0.488281,0.734375,2
117,./DATASET/TreeDetection/valid/images/230507_17...,0.505469,0.582812,0.725781,0.754687,2
898,./DATASET/TreeDetection/train/images/211024_16...,0.765625,0.557813,0.378125,0.514062,2
2384,./DATASET/TreeDetection/train/images/86df265a-...,0.742188,0.460938,0.196094,0.388281,2
343,./DATASET/TreeDetection/train/images/210410_17...,0.36875,0.673438,0.173437,0.43125,2
150,./DATASET/TreeDetection/train/images/1413caff-...,0.421875,0.5,0.84375,1.0,2


### 2.3.3. Обработка датасета Tree 100

In [18]:
dataTree100 = pd.DataFrame(columns=dataCarDetection.columns) 

# Проходимся по всем выборкам
for sample in ['train', 'valid', 'test']:
    labels_name = os.listdir(f'{PATH_TREE_100}{sample}/labels') # Список файлов с разметкой 
    labels = pd.DataFrame(converterYoloToDataFrame(labels_name, f'{PATH_TREE_100}{sample}/labels/', 2)) # Получаем датасет с разметкой
    dataTree100 = pd.concat([dataTree100, labels.T]) # Соединяем выборки в единый датасет

del labels_name, labels # Очищаем память

dataTree100.sample(10) # Посмотрим на данные

  0%|          | 0/82 [00:00<?, ?it/s]

  0%|          | 0/23 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

,image_name,x_center,y_center,w,h,classes
9,./DATASET/Trees100/train/images/arvore03_jpg.r...,0.971094,0.158594,0.022656,0.070312,2
13,./DATASET/Trees100/valid/images/arvore120_jpg....,0.775781,0.690625,0.192188,0.416406,2
45,./DATASET/Trees100/train/images/arvore123_jpg....,0.055469,0.596875,0.110156,0.180469,2
34,./DATASET/Trees100/valid/images/arvore76_jpg.r...,0.48125,0.466406,0.300781,0.661719,2
49,./DATASET/Trees100/train/images/arvore124_jpg....,0.366406,0.582031,0.464844,0.400781,2
140,./DATASET/Trees100/train/images/arvore82_jpg.r...,0.5625,0.348438,0.1,0.239063,2
176,./DATASET/Trees100/train/images/arvore97_jpg.r...,0.530469,0.792188,0.107813,0.415625,2
119,./DATASET/Trees100/train/images/arvore59_jpg.r...,0.500781,0.3375,0.516406,0.614844,2
133,./DATASET/Trees100/train/images/arvore75_jpg.r...,0.667969,0.702344,0.039062,0.055469,2
28,./DATASET/Trees100/train/images/arvore111_jpg....,0.682031,0.386719,0.453906,0.53125,2


## 2.4. Обработка датасета с птицами
### 2.4.1. Обработка датасета с Bird Annotation

In [19]:
dataBirdAnnotation = pd.DataFrame(columns=dataCarDetection.columns) 

# Проходимся по всем выборкам
for sample in ['train', 'valid', 'test']:
    labels_name = os.listdir(f'{PATH_BIRD_ANNOTATION}{sample}/labels') # Список файлов с разметкой 
    labels = pd.DataFrame(converterYoloToDataFrame(labels_name, f'{PATH_BIRD_ANNOTATION}{sample}/labels/', 3)) # Получаем датасет с разметкой
    dataBirdAnnotation  = pd.concat([dataBirdAnnotation, labels.T]) # Соединяем выборки в единый датасет

del labels_name, labels # Очищаем память

# Уменьшаем размер DataFrame
dataBirdAnnotation = datasetMinimization(dataBirdAnnotation, DATASET_DOWNGRADES[print_variable_name(dataBirdAnnotation)])

dataBirdAnnotation.sample(10) # Посмотрим на данные

  0%|          | 0/1362 [00:00<?, ?it/s]

  0%|          | 0/130 [00:00<?, ?it/s]

  0%|          | 0/65 [00:00<?, ?it/s]

There were 1557 photos, now there are 389


,image_name,x_center,y_center,w,h,classes
1127,./DATASET/BirdAnnotation/train/images/descarga...,0.450781,0.614062,0.075,0.109375,3
15916,./DATASET/BirdAnnotation/train/images/voladora...,0.417187,0.196094,0.08125,0.09375,3
4113,./DATASET/BirdAnnotation/train/images/e282_jpg...,0.10625,0.678125,0.046875,0.065625,3
18420,./DATASET/BirdAnnotation/train/images/voladora...,0.489844,0.232031,0.429688,0.204687,3
9636,./DATASET/BirdAnnotation/train/images/voladora...,0.804688,0.760156,0.039062,0.048438,3
1320,./DATASET/BirdAnnotation/train/images/e113_jpg...,0.389844,0.544531,0.057813,0.064062,3
16492,./DATASET/BirdAnnotation/train/images/voladora...,0.057031,0.507812,0.114062,0.109375,3
18307,./DATASET/BirdAnnotation/train/images/voladora...,0.539844,0.696094,0.017188,0.026562,3
2930,./DATASET/BirdAnnotation/train/images/e221_jpg...,0.059375,0.224219,0.0375,0.048438,3
16064,./DATASET/BirdAnnotation/train/images/voladora...,0.417187,0.729688,0.025,0.046875,3


### 2.4.2. Обработка датасета с Birds Detection

In [20]:
dataBirdsDetectod = pd.DataFrame(columns=dataCarDetection.columns) 

# Проходимся по всем выборкам
for sample in ['train', 'valid', 'test']:
    labels_name = os.listdir(f'{PATH_BIRDS_DETECTOR}{sample}/labels') # Список файлов с разметкой 
    labels = pd.DataFrame(converterYoloToDataFrame(labels_name, f'{PATH_BIRDS_DETECTOR}{sample}/labels/', 3)) # Получаем датасет с разметкой
    dataBirdsDetectod = pd.concat([dataBirdsDetectod, labels.T]) # Соединяем выборки в единый датасет

del labels_name, labels # Очищаем память

# Уменьшаем размер DataFrame
dataBirdsDetectod = datasetMinimization(dataBirdsDetectod, DATASET_DOWNGRADES[print_variable_name(dataBirdsDetectod)])

dataBirdsDetectod.sample(10)  # Посмотрим на данные

  0%|          | 0/594 [00:00<?, ?it/s]

  0%|          | 0/170 [00:00<?, ?it/s]

  0%|          | 0/85 [00:00<?, ?it/s]

There were 849 photos, now there are 212


,image_name,x_center,y_center,w,h,classes
664,./DATASET/BirdsDetector/valid/images/e290_jpg....,0.352344,0.402344,0.038281,0.054688,3
1679,./DATASET/BirdsDetector/valid/images/voladoras...,0.058594,0.504687,0.09375,0.0875,3
2882,./DATASET/BirdsDetector/train/images/voladoras...,0.550781,0.379688,0.050781,0.049219,3
3240,./DATASET/BirdsDetector/train/images/voladoras...,0.725781,0.232031,0.076563,0.054688,3
3628,./DATASET/BirdsDetector/train/images/voladoras...,0.765625,0.401562,0.210938,0.180469,3
51,./DATASET/BirdsDetector/train/images/412144_jp...,0.516406,0.489844,0.858594,0.978906,3
2892,./DATASET/BirdsDetector/train/images/voladoras...,0.5,0.688281,0.06875,0.04375,3
3271,./DATASET/BirdsDetector/train/images/voladoras...,0.667188,0.399219,0.054688,0.071094,3
1647,./DATASET/BirdsDetector/valid/images/voladoras...,0.191406,0.360156,0.124219,0.066406,3
4008,./DATASET/BirdsDetector/train/images/voladoras...,0.646875,0.595313,0.4625,0.244531,3


### 2.4.3. Обработка датасета с BirdsV3

In [21]:
dataBirdsV3 = pd.DataFrame(columns=dataCarDetection.columns) 

# Проходимся по всем выборкам
for sample in ['train', 'valid', 'test']:
    labels_name = os.listdir(f'{PATH_BIRDS_V3}{sample}/labels') # Список файлов с разметкой 
    labels = pd.DataFrame(converterYoloToDataFrame(labels_name, f'{PATH_BIRDS_V3}{sample}/labels/', 3)) # Получаем датасет с разметкой
    dataBirdsV3 = pd.concat([dataBirdsV3, labels.T]) # Соединяем выборки в единый датасет

del labels_name, labels # Очищаем память

# Уменьшаем размер DataFrame
dataBirdsV3 = datasetMinimization(dataBirdsV3, DATASET_DOWNGRADES[print_variable_name(dataBirdsV3)])

dataBirdsV3.sample(10)  # Посмотрим на данные

  0%|          | 0/582 [00:00<?, ?it/s]

  0%|          | 0/55 [00:00<?, ?it/s]

  0%|          | 0/28 [00:00<?, ?it/s]

There were 665 photos, now there are 332


,image_name,x_center,y_center,w,h,classes
1809,./DATASET/BirdsV3/train/images/birds-on-roof_j...,0.217188,0.820312,0.009375,0.035937,3
827,./DATASET/BirdsV3/train/images/655b0416-7147-4...,0.363281,0.532031,0.426563,0.532813,3
1710,./DATASET/BirdsV3/train/images/birds-on-roof_j...,0.230469,0.764844,0.015625,0.0375,3
3766,./DATASET/BirdsV3/train/images/pigeons-roof-12...,0.669531,0.507812,0.0375,0.067187,3
1844,./DATASET/BirdsV3/train/images/birds-roof-1271...,0.498437,0.55625,0.039062,0.051562,3
3105,./DATASET/BirdsV3/train/images/photo-156466976...,0.448437,0.446094,0.076563,0.073438,3
3512,./DATASET/BirdsV3/train/images/pigeons-on-an-o...,0.667188,0.566406,0.126562,0.121875,3
841,./DATASET/BirdsV3/train/images/7912165756_d49b...,0.566406,0.7375,0.014063,0.020313,3
3442,./DATASET/BirdsV3/train/images/pigeonroofdamag...,0.475781,0.223438,0.064062,0.085938,3
2487,./DATASET/BirdsV3/train/images/DSC_0597_jpg.rf...,0.602344,0.154688,0.01875,0.03125,3


## 2.5. Обработка фотографий со зданиями
### 2.5.1. Обработка датасета Building Detection 

In [22]:
dataBuildingDetection = pd.DataFrame(columns=dataCarDetection.columns) 

# Проходимся по всем выборкам
for sample in ['train', 'valid', 'test']:
    labels_name = os.listdir(f'{PATH_BUILDING_DETECTION}{sample}/labels') # Список файлов с разметкой 
    labels = pd.DataFrame(converterYoloToDataFrame(labels_name, f'{PATH_BUILDING_DETECTION}{sample}/labels/', 4)) # Получаем датасет с разметкой
    dataBuildingDetection = pd.concat([dataBuildingDetection, labels.T]) # Соединяем выборки в единый датасет

del labels_name, labels # Очищаем память

# Уменьшаем размер DataFrame
dataBuildingDetection = datasetMinimization(dataBuildingDetection, DATASET_DOWNGRADES[print_variable_name(dataBuildingDetection)])

dataBuildingDetection.sample(10)  # Посмотрим на данные

  0%|          | 0/726 [00:00<?, ?it/s]

  0%|          | 0/47 [00:00<?, ?it/s]

  0%|          | 0/29 [00:00<?, ?it/s]

There were 802 photos, now there are 401


,image_name,x_center,y_center,w,h,classes
788,./DATASET/BuildingDetection/train/images/92_A_...,0.498798,0.498798,0.997596,0.997596,4
47,./DATASET/BuildingDetection/valid/images/84_A_...,0.168269,0.449519,0.336538,0.882212,4
591,./DATASET/BuildingDetection/train/images/361_A...,0.498798,0.498798,0.997596,0.997596,4
525,./DATASET/BuildingDetection/train/images/30_A_...,0.406791,0.50179,0.677789,0.845116,4
408,./DATASET/BuildingDetection/train/images/231_A...,0.36899,0.694712,0.737981,0.605769,4
580,./DATASET/BuildingDetection/train/images/359_A...,0.498798,0.5,0.997596,1.0,4
660,./DATASET/BuildingDetection/train/images/399_A...,0.498798,0.5,0.997596,1.0,4
37,./DATASET/BuildingDetection/train/images/109_A...,0.416136,0.635896,0.825559,0.723401,4
205,./DATASET/BuildingDetection/train/images/164_A...,0.498798,0.5,0.997596,1.0,4
494,./DATASET/BuildingDetection/train/images/300_A...,0.498798,0.498798,0.997596,0.997596,4


### 2.5.2. Обработка датасета Josh Edits 

In [23]:
dataJoshEdits = pd.DataFrame(columns=dataCarDetection.columns) 

# Проходимся по всем выборкам
for sample in ['train', 'valid', 'test']:
    labels_name = os.listdir(f'{PATH_JOSH_EDITS}{sample}/labels') # Список файлов с разметкой 
        
    labels = pd.DataFrame(converterYoloToDataFrame(labels_name, f'{PATH_JOSH_EDITS}{sample}/labels/', 4)) # Получаем датасет с разметкой
    dataJoshEdits = pd.concat([dataJoshEdits, labels.T]) # Соединяем выборки в единый датасет

del labels_name, labels # Очищаем память

# Уменьшаем размер DataFrame
dataJoshEdits = datasetMinimization(dataJoshEdits, DATASET_DOWNGRADES[print_variable_name(dataJoshEdits)])

dataJoshEdits.sample(10)  # Посмотрим на данные

  0%|          | 0/3036 [00:00<?, ?it/s]

  0%|          | 0/290 [00:00<?, ?it/s]

  0%|          | 0/139 [00:00<?, ?it/s]

There were 3251 photos, now there are 1625


,image_name,x_center,y_center,w,h,classes
4948,./DATASET/JoshEdits/train/images/sv_42-1988112...,0.180469,0.442188,0.203906,0.1,4
198,./DATASET/JoshEdits/test/images/sv_42-16683047...,0.095312,0.464062,0.189844,0.253906,4
2473,./DATASET/JoshEdits/train/images/image_84_png....,0.617969,0.5,0.764062,1.0,4
1692,./DATASET/JoshEdits/train/images/debug_image_2...,0.028906,0.470313,0.057031,0.128906,4
226,./DATASET/JoshEdits/valid/images/image_67_png....,0.292969,0.513281,0.489844,0.642188,4
5009,./DATASET/JoshEdits/train/images/sv_42-2009658...,0.553906,0.467187,0.376563,0.34375,4
491,./DATASET/JoshEdits/train/images/37_4044618508...,0.110937,0.375781,0.221875,0.520312,4
4360,./DATASET/JoshEdits/train/images/sv_41-9362616...,0.810156,0.364063,0.379688,0.5,4
212,./DATASET/JoshEdits/test/images/sv_42-20137504...,0.050781,0.442969,0.100781,0.071875,4
4048,./DATASET/JoshEdits/train/images/sv_37-403977_...,0.809375,0.413281,0.3625,0.304688,4


# 3. Объединение датасетов в единый датасет

In [24]:
data = pd.concat(
    [
        dataCarDetection, dataCarPerson, dataCarsDetection, # Авто
        dataSTN, dataUnilityPole, dataTransmission, # ЛЭП
        dataTree, dataTreeDetection, dataTree100, # Деревья
        dataBirdAnnotation, dataBirdsDetectod, dataBirdsV3, # Птицы
        dataBuildingDetection, dataJoshEdits # Здания
    ], 
    ignore_index=True
) # Объеденим датасеты в один единый

del dataCarDetection, dataCarPerson, dataCarsDetection # очищаем память от датасетов с Авто
del dataSTN, dataUnilityPole, dataTransmission # очищаем память от датасетов с ЛЭП
del dataTree, dataTreeDetection, dataTree100 # очищаем память от датасетов со Деревьями
del dataBirdAnnotation, dataBirdsDetectod, dataBirdsV3 # очищаем память от датасетов со Птицами
del dataBuildingDetection, dataJoshEdits # очищаем память от датасетов со Зданиями

data.sample(10) # Посмотрим на данные

,image_name,x_center,y_center,w,h,classes
19355,./DATASET/TreeDetection/train/images/7fd5d836-...,0.527344,0.427344,0.878125,0.826562,2
26395,./DATASET/BirdsV3/train/images/83ff5a3360c7b39...,0.946094,0.471094,0.045312,0.051562,3
4737,./DATASET/CarPerson/Car-Person-v2-Roboflow-Owa...,0.399038,0.520433,0.217548,0.19351,0
10963,./DATASET/Transmission/data_original_size_v1/d...,0.876953,0.876953,0.0,0.999023,5
2200,./DATASET/CarPerson/Car-Person-v2-Roboflow-Owa...,0.628606,0.74399,0.115385,0.05649,0
4985,./DATASET/CarPerson/Car-Person-v2-Roboflow-Owa...,0.96875,0.566106,0.038462,0.054087,0
20675,./DATASET/BirdAnnotation/train/images/cuidados...,0.692969,0.494531,0.034375,0.051562,3
20648,./DATASET/BirdAnnotation/train/images/birds-pi...,0.625,0.9,0.067187,0.096875,3
6968,./DATASET/UtilityPole/train/images/images--10-...,0.84976,0.728365,0.137019,0.408654,1
13194,./DATASET/Transmission/data_original_size_v1/d...,0.633203,0.633203,0.733073,0.09191,5


In [25]:
data['image_name'].unique().shape # Количество фотографий

(7105,)

In [26]:
# Изменим тип данных координат с string на float
for column in data.columns[1:-1]:
    data[column] = data[column].astype(float)

data['classes'] = data['classes'].astype(int) # Изменим тип классов с string на int
data.info() # Посмотрим на сведение дата сета

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32381 entries, 0 to 32380
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   image_name  32381 non-null  object 
 1   x_center    32381 non-null  float64
 2   y_center    32381 non-null  float64
 3   w           32381 non-null  float64
 4   h           32381 non-null  float64
 5   classes     32381 non-null  int32  
dtypes: float64(4), int32(1), object(1)
memory usage: 1.4+ MB


In [27]:
count_image = data['image_name'].unique().shape[0]
print(f'Dataset contains {count_image} unique photos and {data.shape[0]} objects')
del count_image # Очищаем память

Dataset contains 7105 unique photos and 32381 objects


    0 - машина
    1 - ЛЭП
    2 - Деревья
    3 - Птицы
    4 - Дом/здание
    5 - Провода

In [28]:
# Выведем в процентах количество уникальных изображений по классам
data.groupby('image_name')['classes'].head().value_counts().sort_index() /  data.groupby('image_name')['classes'].head().shape[0] * 100

classes
0    21.352986
1     3.935860
2    13.972573
3    14.755426
4    20.111219
5    25.871936
Name: count, dtype: float64

    0 - машина
    1 - ЛЭП
    2 - Деревья
    3 - Птицы
    4 - Дом/здание
    5 - Провода

In [29]:
# Распределение классов среди всех объектов
data['classes'].value_counts().sort_index() / data.shape[0] * 100

classes
0    20.351441
1     2.714555
2     8.702634
3    25.518051
4    11.599395
5    31.113925
Name: count, dtype: float64

In [30]:
# Разделим данные на выборки
train_name_file, test_name_file = train_test_split(data['image_name'].unique(), test_size=.30, random_state=42) 
valid_name_file, test_name_file = train_test_split(test_name_file, test_size=.33, random_state=42)

# Посмотрим на процентное отношение выборок 
pd.DataFrame([len(train_name_file), len(valid_name_file), len(test_name_file)], columns=['size']) / len(data['image_name'].unique()) * 100

,size
0,69.992963
1,20.098522
2,9.908515


In [31]:
train_dataCarDetection = data[data['image_name'].isin(train_name_file)] # Записи для тренировочных данных
valid_dataCarDetection = data[data['image_name'].isin(valid_name_file)] # Записи для валидационных данных
test_dataCarDetection = data[data['image_name'].isin(test_name_file)] # Записи для тестовых данных
del data, train_name_file, valid_name_file, test_name_file # Очистим память

In [32]:
import os
list_paths_to_create = [
    'DATA', 'DATA/train', 
    'DATA/valid', 'DATA/test', 'DATA/train/images', 
    'DATA/train/labels', 'DATA/valid/images', 'DATA/valid/labels',
    'DATA/test/images', 'DATA/test/labels'
]

for i in list_paths_to_create:
    os.mkdir(i)


In [33]:
def copyFileDataSet(dataCarDetection, pathEnd):
    '''
        Функция позволяет скопировать фото в новую папку и под каждую из них создать txt файл с разметкой
        Input:
            dataCarDetection - DataFrame содержащий в себе следующиго типа:
                Признаки    Non-Null?  Тип
                image_name  non-null   object 
                x_center    non-null   float64
                y_center    non-null   float64
                w           non-null   float64
                h           non-null   float64
                classes     non-null   int64
            pathEnd - путь созданной папки, внутри которой созданы папки
                images и labels
    '''
    nameFiles = dataCarDetection['image_name'].unique() # Получаем список уникальных файлов

    # Запускаем цикл по все файлам
    for item in tqdm(nameFiles):

        # shutil.copy2(item, pathEnd+'/images') # Копируем данный файл в другую директорию/
        img = cv2.imread(item, cv2.IMREAD_UNCHANGED)
        img = cv2.resize(cv2.UMat(img), (1280, 720), cv2.INTER_NEAREST)
        item = item.replace('\\', '/')
        pathEnd = pathEnd.replace('\\', '/')
        
        cv2.imwrite(pathEnd + '/images/' + item.split('/')[-1], img)
        
        dataList = list(dataCarDetection[dataCarDetection['image_name'] == item].values) # Получаем все объекты на фото
        labelsFile = open(pathEnd + '/labels/' + item.split('/')[-1][:-3] + 'txt', 'w+') # Создаём текстовый документ с разметкой
        isFirst = True # Флаг первой строки

        # Запускаем цикл по всем объектам на фото
        for data in dataList:

            # Получаем линию с данными 
            line = '' if isFirst else '\n'
            isFirst = False
            line += str(data[-1]) + ' ' + ' '.join(str(number) for number in data[1:-1]) 
            
            labelsFile.write(line) # Записываем в файл
        labelsFile.close() # Закрываем файл
        
copyFileDataSet(train_dataCarDetection, './DATA/train') # Формируем тренировочную выборку
copyFileDataSet(valid_dataCarDetection, './DATA/valid') # Формируем валидационную выборку
copyFileDataSet(test_dataCarDetection, './DATA/test') # Формируем тестовую выборку
del train_dataCarDetection, valid_dataCarDetection, test_dataCarDetection # Очищаем память

  0%|          | 0/4973 [00:00<?, ?it/s]

  0%|          | 0/1428 [00:00<?, ?it/s]

  0%|          | 0/704 [00:00<?, ?it/s]

In [34]:
# Глобальный путь до директории с проектом 
global_path = os.path.abspath('.') 
yolo_format = dict(
    path=f'{global_path}/DATA',
    train='../train',
    val='../valid',
    test='../test',       
    nc=6,
    names={
        0: 'Car',
        1: 'STN',
        2: 'Tree',
        3: 'Bird',
        4: 'House',
        5: 'Cable',
    }
)

del global_path # Очищаем память
             
with open('./DATA/yolo.yaml', 'w') as outfile:
    yaml.dump(yolo_format, outfile, default_flow_style=False)

In [2]:
epochTraning = 250 # Количество эпох обучения
pathYalm = "./DATA/yolo.yaml" # Путь до файла yalm
patience = 30 # Терпимость эпох для переобучения
imgsz = 1280 # Размер изображения

In [3]:
model = YOLO('yolov8n.yaml').load('yolov8n.pt')
model.train(
    data=pathYalm, 
    epochs=epochTraning, 
    patience=patience, 
    imgsz=imgsz,
    device=0
) 

Transferred 355/355 items from pretrained weights
Ultralytics YOLOv8.1.47  Python-3.9.6 torch-2.1.0+cu121 CUDA:0 (NVIDIA GeForce RTX 3080, 10240MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.yaml, data=./DATA/yolo.yaml, epochs=250, time=None, patience=30, batch=16, imgsz=1280, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save

train: Scanning D:\MyProjects\Хакатоны\TerraWing\src\neural_network\DATA\train\labels.cache... 4973 images, 873 backgro

train: WARNING  D:\MyProjects\\TerraWing\src\neural_network\DATA\train\images\image_000001036_jpg.rf.3eb241105e26834305cefe3448dc4f6e.jpg: 1 duplicate labels removed



val: Scanning D:\MyProjects\Хакатоны\TerraWing\src\neural_network\DATA\valid\labels.cache... 1428 images, 241 backgroun


Plotting labels to runs\detect\train\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 1280 train, 1280 val
Using 8 dataloader workers
Logging results to runs\detect\train
Starting training for 250 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all       1428       4552      0.774      0.723      0.756      0.505
EarlyStopping: Training stopped early as no improvement observed in last 30 epochs. Best results observed at epoch 197, best model saved as best.pt.
To update EarlyStopping(patience=30) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



227 epochs completed in 9.589 hours.
Optimizer stripped from runs\detect\train\weights\last.pt, 6.3MB
Optimizer stripped from runs\detect\train\weights\best.pt, 6.3MB

Validating runs\detect\train\weights\best.pt...
Ultralytics YOLOv8.1.47  Python-3.9.6 torch-2.1.0+cu121 CUDA:0 (NVIDIA GeForce RTX 3080, 10240MiB)
YOLOv8n summary (fused): 168 layers, 3006818 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 45/45 [00:12


                   all       1428       4552      0.787      0.714      0.759      0.507
                   Car       1428       1582      0.653      0.419      0.472      0.252
                   STN       1428        118      0.884      0.965      0.979      0.828
                  Tree       1428        540      0.802      0.731      0.777      0.448
                  Bird       1428       1569      0.738      0.612      0.672      0.389
                 House       1428        743      0.858      0.844      0.894       0.62
Speed: 0.3ms preprocess, 1.7ms inference, 0.0ms loss, 1.4ms postprocess per image
Results saved to runs\detect\train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000014C4B391190>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
    

In [ ]:
model = YOLO('yolov5n6.yaml').load('yolov5n6.pt')
model.train(
    data=pathYalm, 
    epochs=epochTraning, 
    patience=patience, 
    imgsz=imgsz,
    device=0
)

WARNING  Ultralytics YOLO P6 models now use -p6 suffix. Renaming yolov5n6 to yolov5n-p6.
PRO TIP  Replace 'model=yolov5n6.pt' with new 'model=yolov5n6u.pt'.
YOLOv5 'u' models are trained with https://github.com/ultralytics/ultralytics and feature improved performance vs standard YOLOv5 models trained with https://github.com/ultralytics/yolov5.

Transferred 563/563 items from pretrained weights
Ultralytics YOLOv8.1.47  Python-3.9.6 torch-2.1.0+cu121 CUDA:0 (NVIDIA GeForce RTX 3080, 10240MiB)
engine\trainer: task=detect, mode=train, model=yolov5n6.yaml, data=./DATA/yolo.yaml, epochs=250, time=None, patience=30, batch=16, imgsz=1280, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dro

train: Scanning D:\MyProjects\Хакатоны\TerraWing\src\neural_network\DATA\train\labels.cache... 4973 images, 873 backgro

train: WARNING  D:\MyProjects\\TerraWing\src\neural_network\DATA\train\images\image_000001036_jpg.rf.3eb241105e26834305cefe3448dc4f6e.jpg: 1 duplicate labels removed



val: Scanning D:\MyProjects\Хакатоны\TerraWing\src\neural_network\DATA\valid\labels.cache... 1428 images, 241 backgroun


Plotting labels to runs\detect\train2\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 91 weight(decay=0.0), 100 weight(decay=0.0005), 99 bias(decay=0.0)
Image sizes 1280 train, 1280 val
Using 8 dataloader workers
Logging results to runs\detect\train2
Starting training for 250 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     44/250        11G      1.239      1.105      1.323         39       1280: 100%|██████████| 311/311 [02:14<00:00,  
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 45/45 [00:25

                   all       1428       4552      0.753      0.645      0.733      0.481



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     45/250      11.4G      1.242      1.118      1.327         76       1280: 100%|██████████| 311/311 [02:07<00:00,  
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  71%|███████   | 32/45 [00:24